In [11]:
import string
import re
import os
import copy
import pickle
import math
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
lemmatizer = WordNetLemmatizer()

In [12]:
Picklefile1 = open('dictionary_high_postings', 'rb')
dict_high_posting = pickle.load(Picklefile1)

Picklefile2 = open('dictionary_low_postings', 'rb')
dict_low_posting = pickle.load(Picklefile2)

Picklefile3 = open('dictionary_counters', 'rb')
dict_counters = pickle.load(Picklefile3)

Picklefile4 = open('unique_words_corpus', 'rb')
list_unique_words_corpus = pickle.load(Picklefile4)
len_list_unique_words_corpus = len(list_unique_words_corpus)

Picklefile5 = open('document_names_all', 'rb')
all_doc_ids = pickle.load(Picklefile5)
len_all_doc_ids = len(all_doc_ids)

Picklefile6 = open('idf_dictionary', 'rb')
dict_idf = pickle.load(Picklefile6)

Picklefile7 = open('doc_quality_scores', 'rb')
dict_quality_score = pickle.load(Picklefile7)

Picklefile8 = open('nt_maximum', 'rb')
max_nt = pickle.load(Picklefile8)

In [13]:
print("Enter the query:")
query  = input()
print("Enter the value of k:")
k_value = int(input())
#Applying text pre-processing on the query also
text = query
text = text.lower()
text = re.sub(r'\S+@\S+', ' ', text)
text = re.sub(r'[a-zA-Z]+[0-9]+', '', text)
text = re.sub(r'[0-9]+[a-zA-Z]+', '', text)
translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
text = text.translate(translator)
word_tokens = word_tokenize(text)
stop_words = set(stopwords.words("english"))
text = [word for word in word_tokens if word not in stop_words]
lemmas = [lemmatizer.lemmatize(word) for word in text]
list_query_tokens = copy.deepcopy(lemmas)
# print(list_query_tokens)

Enter the query:
BMW with the rear wheel spinning wildly and someone groping for the kill switch
Enter the value of k:
10


In [14]:
#Now for each query term extracting the posting list
list_query_postings = []
for tk in list_query_tokens:
    list_post = dict_high_posting.get(tk)
    if list_post is not None:
        list_query_postings.extend(list_post)

set_query_postings = set(list_query_postings)
list_retrieved_postings = list(set_query_postings)

dict_counts_query_tokens = Counter(list_query_tokens)
list_query_tokens_freq = list(dict_counts_query_tokens.values())
max_freq_query = max(list_query_tokens_freq)

vector_query = np.zeros(len_list_unique_words_corpus)
for i in range(len_list_unique_words_corpus):
    wrd = list_unique_words_corpus[i]
    idf_val = dict_idf[wrd]
    term_freq = dict_counts_query_tokens.get(wrd)
    if term_freq is None:
        term_freq = 0
    #tf_val = math.log10(1 + term_freq)
    tf_val = 0.5 + 0.5 * (term_freq / max_freq_query)
    val_product = idf_val * tf_val
    vector_query[i] = val_product

# print(vector_query)
# ct = 0
# for j in range(len_list_unique_words_corpus):
#     v = vector_query[j]
#     if v != 0:
#         ct += 1
#         print(list_unique_words_corpus[j])
# print(ct)

In [15]:
dict_net_scores = {}
list_pairs_net_scores = []

for doc in list_retrieved_postings:
    vector_doc = np.zeros(len_list_unique_words_corpus)
    val_gd = dict_quality_score[doc]
    curr_dict = dict_counters[doc]
    list_doc_tokens_freq = list(curr_dict.values())
    max_freq_doc = max(list_doc_tokens_freq)
    
    for i in range(len_list_unique_words_corpus):
        wrd = list_unique_words_corpus[i]
        idf_val = dict_idf[wrd]
        term_freq = curr_dict.get(wrd)
        if term_freq is None:
            term_freq = 0
#         tf_val = math.log10(1 + term_freq)
        tf_val = 0.5 + 0.5 * (term_freq / max_freq_doc)
        val_product = idf_val * tf_val
        vector_doc[i] = val_product
    
    norm_q_vector = np.linalg.norm(vector_query)
    norm_doc_vector = np.linalg.norm(vector_doc)
    cosine_sim = (np.dot(vector_query, vector_doc)) / (norm_q_vector * norm_doc_vector)
    net_score = val_gd + cosine_sim
    pair = [doc, net_score]
    list_pairs_net_scores.append(pair)

list_pairs_net_scores.sort(key= lambda x:x[1], reverse=True)

len_final_retrieved = 0
if len(list_pairs_net_scores) < k_value:
    len_final_retrieved = len(list_pairs_net_scores)
else:
    len_final_retrieved = k_value


list_final_retrieved = []

for ab in range(len_final_retrieved):
    list_final_retrieved.append(list_pairs_net_scores[ab])

print(f"The top {len_final_retrieved} documents in decreasing order of net scores are")


for ad in range(len_final_retrieved):
    print(f"{ad+1}.  {list_final_retrieved[ad][0]}\t\t{list_final_retrieved[ad][1]}")

The top 10 documents in decreasing order of net scores are
1.  rec.motorcycles\105136		1.3265085745609821
2.  rec.autos\103243		1.3060983927387824
3.  talk.politics.mideast\77207		1.2856906341754577
4.  rec.autos\102836		1.285686475407116
5.  comp.windows.x\66871		1.2652817084490806
6.  rec.autos\102770		1.265275611351158
7.  rec.autos\102816		1.265273047306603
8.  alt.atheism\53343		1.265272228004764
9.  rec.motorcycles\105237		1.2448765254364398
10.  rec.motorcycles\104844		1.2448760386455795
